## Projektsökvägar
Sätter nödvändiga sökvägar som kommer att användas genom hela skriptets gång.

In [2]:
### MAGNUS LOKALT ###
import os
from pathlib import Path
prj_dir = Path(r"C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01")
LGs_info_name = 'Ledningar_SKB_2023_11_01.txt'
LGs_info_path = os.path.join(prj_dir, LGs_info_name)
gdb_name = 'SKB_2023_11_01.gdb'
gdb = os.path.join(prj_dir, gdb_name)
serv_prj_dir = Path(r"Z:\Projekt\Analys_2023\ledningar")
cvd_LEDNINGSGATA = 'cvd_LEDNINGSGATA.txt'
cvd_LEDNINGSGATA_path = os.path.join(prj_dir, cvd_LEDNINGSGATA)
cvd_DATE = 'cvd_DATE_2023.txt'
cvd_DATE_path = os.path.join(prj_dir, cvd_DATE)
all_merged_name = "all_2023_11_01"
version = "2023_11_01"

In [2]:
### KARIN LOKALT ###
import os
from pathlib import Path
prj_dir = Path(r"C:\SVK_2024")

# Namn på styrfil med information om de ledningar som ska köras:
LGs_info_name = 'veg_kantträd_241111.txt'
LGs_info_path = os.path.join(r"Q:\Projekt\Data_2024\styrfiler", LGs_info_name)

# Namn på den gdb där data ska läggas. Den måste finnas och ha domäner inlagda. 
# Mall att kopiera finns i GitHub-repot under python\gdb-mallar-och-domänkoder.
gdb_name = 'veg_kantträd_241111.gdb'
gdb = os.path.join(prj_dir, gdb_name)

# uppdragsmappens mapp "ledningar" där alla terrascan-resultat hamnar:
serv_prj_dir = Path(r"Q:\Projekt\Analys_2024\ledningar")

# domäner som används:
cvd_LEDNINGSGATA = 'cvd_LEDNINGSGATA.txt'
cvd_LEDNINGSGATA_path = os.path.join(r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2024\30047770-SVK-Analys-av-laserdata-extern\python\gdb-mallar-och-domänkoder", 
                                     cvd_LEDNINGSGATA)
cvd_DATE = 'cvd_DATE_2023.txt'
cvd_DATE_path = os.path.join(r"C:\Users\SEKNNO\OneDrive - Sweco AB\SVK_2024\30047770-SVK-Analys-av-laserdata-extern\python\gdb-mallar-och-domänkoder", 
                             cvd_DATE)


all_merged_name = "2024_11_11"
version = "2024_11_11"

# Merge SKB text files for all blocks of a powerline
Kör blockskript i respektive servermapp för att ha den absolut senaste uppdaterade.

In [8]:
import os
import sys
import glob
import fileinput
from pathlib import Path
#sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata")
import SKB
import SVK
import pandas as pd

def combine_blocks(row):
    LG = row[0]
    line = row[1]
    line_dir = serv_prj_dir / LG / f"line_{line}"
    
    print(f"Doing {LG}_{line}")
    block_dir = os.path.join(line_dir, "kantträd", "block")
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge files for all blocks into one
    SVK.merge_blocks(block_dir, "*.txt", combined_blocks_path)

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
print(LGs_info)
LGs_info.apply(combine_blocks, axis=1)

      LG  line Littera  Spänning    Strömtyp  Luftledning               Namn  \
0  LG700     1    RL31       220  Växelström  Luftledning  Anneberg-Danderyd   
1  LG700     2    RL32       220  Växelström  Luftledning  Anneberg-Danderyd   

      Status Ingår  Kommentar  
0  Befintlig    Ja          0  
1  Befintlig    Ja          0  
Doing LG700_1
Doing LG700_2


0    None
1    None
dtype: object

## Kopiera skb_raw.txt
Denna kopierar över alla skb_raw.txt till den temporära mappen på skrivbordet

In [9]:
import os
import filecmp
import shutil
import pandas as pd
df = pd.DataFrame()
to_dir = os.path.join(prj_dir, "ledningar")
from_dir = serv_prj_dir

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
from_dir_SKB = []
for i, j in zip(LGs_info["LG"], LGs_info["line"]):
    from_dir_SKB.append(f'{i}\\line_{j}\\Kantträd')
to_dir_SKB = []
for i, j in zip(LGs_info["LG"], LGs_info["line"]):
    to_dir_SKB.append(f'{i}\\line_{j}\\SKB')

outlist_to_dir = []
for i in to_dir_SKB:
    outlist_to_dir.append((f"{to_dir}\{i}\SKB_raw.txt").replace('\\\\', '\\'))

outlist_from_dir = []
for i in from_dir_SKB:
    outlist_from_dir.append(
        (f"{from_dir}\{i}\SKB_raw.txt").replace('\\\\', '\\'))

for i, j in zip(outlist_from_dir, outlist_to_dir):
    if os.path.exists(j.replace('\\\\', '\\')):
        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))
    else:
        folder1 = j.split("\\")[-4]
        folder2 = j.split("\\")[-3]
        folder3 = j.split("\\")[-2]
        os.makedirs(
            os.path.join(
                to_dir,
                folder1,
                folder2,
                folder3,
                #"SKB"
            )
        )
        shutil.copyfile(i.replace('\\\\', '\\'), j.replace('\\\\', '\\'))

for i, j in zip(outlist_from_dir, outlist_to_dir):
    print(filecmp.cmp(i, j))


True
True


# Calculate distances to wire for trees

In [11]:
import sys
import os, SKB, time, arcpy
from time import gmtime, strftime
from pathlib import Path
import pandas as pd

# Featuredataset för kantträdspunkter innehållande X, Y och markens Z-värde under trädet:
SKB_XYmZ_location = os.path.join(gdb, 'SKB_XYmZ')

# Featuredataset för kantträdspunkter innehållande X, Y och Z-värde för trädkronans topp:
SKB_XYZ_location = os.path.join(gdb, 'SKB_XYZ')


# Featuredataset för faslinor, som i ett tidigare steg lästs in från DGN-filerna (Wires.ipynb):
wires_fd_1 = os.path.join(gdb, 'Wires')
# wires_fd_2 = os.path.join(gdb, 'Wires_incomplete') # Troligen en rest från när vi hade dåliga punktmoln som orsakade luckor i faslinorna
wires_suffix = '_fas'
search_radius = 100 # How far from the point should Near3D search for wires
SKB_template = os.path.join(gdb, 'SKB_template') # Detta bör vara en mall för featureklassen med resultatet som ska skapas
atgardsar = 2024 # TODO: BÖR INTE VARA MED LÄNGRE, KOLLA SÅ ATT DET INTE ANVÄNDS LÄNGRE NER I SKRIPTET
leverantor = 'SWECO'
insamlingsmetod = 20
matosakerhet_p = 1000
matosakerhet_h = 1000
arcpy.env.overwriteOutput = True

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)


def compute_SKB_distances(row):
    LG = row[0] # TODO: Hänvisa till kolumnnamn i styrfilen istället för index
    line = row[1] # TODO: Hänvisa till kolumnnamn i styrfilen istället för index
    line_dir = prj_dir / "ledningar" / LG / f"line_{line}"
    SKB_dir = line_dir / "SKB" 
    
    arcpy.env.overwriteOutput = True
    print(f"Starting with {LG}_{line}")
    
    # Namn på featureklasser som ska skapas i featuredataset som definierats i början av skriptet:
    SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    akuta_trad = f"{LG}_{line}_akuta"
    ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    # Sökvägar till featureklasserna:
    SKB_XYmZ_path = os.path.join(SKB_XYmZ_location, SKB_XYmZ)
    SKB_XYZ_path = os.path.join(SKB_XYZ_location, SKB_XYZ)
    akuta_trad_path = os.path.join(SKB_XYZ_location, akuta_trad) # Används ej? Inaktuellt eftersom vi inte längre delar upp kantträd i akuta och ej akuta
    ej_akuta_trad_path = os.path.join(SKB_XYZ_location, ej_akuta_trad) # Används ej? Inaktuellt eftersom vi inte längre delar upp kantträd i akuta och ej akuta

    wires_1 = os.path.join(wires_fd_1, f"{LG}_{line}{wires_suffix}")
    # wires_2 = os.path.join(wires_fd_2, f"{LG}_{line}{wires_suffix}") # REST SOM INTE LÄNGRE SKA VARA MED? DEN FANNS NÄR VI HADE DÅLIGA PUNKTMOLN SOM ORSAKADE LUCKOR I FASLINORNA
    
    if arcpy.Exists(wires_1):
        wires = wires_1
        print(f"{LG}_{line} faslinor i Wires")
    #elif arcpy.Exists(wires_2):
    #    wires = wires_2
    #    print(f"{LG}_{line} faslinor i Wires_incomplete")
    else:
        print(f"Inga faslinor för {LG}_{line}")
        return
    
    # Create an fc for X,Y,Z,dZ,mZ with mZ as Z coordinate
    # Z är trädtoppens Z-värde i RH2000 dvs höjd över havet
    # dZ är skillanden mellan markens Z-värde under trädet och trädkronans Z-värde
    # mZ är markens Z-värde under trädet
    # Output från TerraScan innehåller X, Y, Z och dZ.
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    SKB.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    print(f'{LG}_{line}_SKB_XYmZ created')

    # Call arcpy function Near3D to calculate distance from mZ to wire, 
    # on SKB_XYmZ
    SKB.dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    print(f'dist_mZ_wire calculated for {LG}_{line}')
    print(f'{strftime("%H:%M:%S", gmtime())}')

    # Create feature class for SKB where point XYZ is the XYZ for the tree tops
    SKB.create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)

    # Fill SKB_XYZ with values from SKB_XYmZ and set Z coordinate to Z (instead of mZ)
    LG_nr = int(LG[-3:])
    SKB.populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod,
                        matosakerhet_p, matosakerhet_h)
    print('SKB_XYZ populated')  

    # NEAR_DIST computed by Near3D, and then used as AvstHori in populate_SKB_XYZ,
    # does not seem to be the perpendicular distance to the wire, but rather the 
    # distance to the nearest point on the wire (due to sag, I guess)
    # So: Use NEAR analysis in 2D to create a new NEAR column, then use that one as AvstHori
    SKB.compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    print('Avstand fas computed')
          
    # Now replace old avst_hori with new, correct one from compute_horizontal_dist. 
    # Then remove the NEAR-fields.
    SKB.update_horizontal_dist(SKB_XYZ_path)
    print('Avstand horisontellt computed')
     
    print("Setting ground Z for XYZ points")
    all_fc = os.path.join(SKB_XYZ_path, SKB_XYZ)
    SKB.update_z_coordinate(SKB_XYZ_location, SKB_XYZ)

    
          
    print(f"Done with {LG}_{line}")
# Coordinate system
sr = sweref99TM()

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(compute_SKB_distances, axis=1)
    


Starting with LG700_1
Inga faslinor för LG700_1
Starting with LG700_2
Inga faslinor för LG700_2


0    None
1    None
dtype: object

## Städa bland fält och fältnamn
## VARFÖR TAS FÄLTEN LEDNINGSGATA OCH INSAMLINGSMETOD BORT? DE LÄGGS SEDAN TILL  IGEN. HAR DE FEL FORMAT I MALLDATABASEN? BÄTTRE ATT ÄNDRA DET I SÅ FALL.


In [ ]:
import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

SKB_XYZ_fd = os.path.join(prj_dir, gdb ,"SKB_XYZ")                                                                                            

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    
    fieldnames = ["_SKB_XYZ"]
    
    for fieldname in fieldnames:
        
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
        print(f"Altering fields for {LG}_{line}{fieldname}")
        path_joined = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
        field_names = [f.name for f in arcpy.ListFields(path_joined)]
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if "LEDNINGSGATA" in field_names:    
            arcpy.management.DeleteField(
                path_joined,
                "LEDNINGSGATA")
        if "INSAMLINGSMETOD" in field_names:    
            arcpy.management.DeleteField(
                path_joined,
                "INSAMLINGSMETOD")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG040_1_SKB_XYZ
Altering fields for LG040_2_SKB_XYZ
Altering fields for LG040_3_SKB_XYZ
Altering fields for LG040_4_SKB_XYZ
Altering fields for LG040_5_SKB_XYZ
Altering fields for LG040_6_SKB_XYZ
Altering fields for LG661_1_SKB_XYZ
Altering fields for LG661_2_SKB_XYZ


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

## Lägga till statisk information
Lägger till statisk information som kommer att vara likadan för alla kantträd som körs.<br>
Lägger till Ursprung, inscanningsmetod, mätosäkerhet i plan och höjd samt leverans datum

In [9]:
# Add data from LEDNIGNAR.txt
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

SKB_location = os.path.join(gdb, 'SKB_XYZ')

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    print(line)
    print("Klart")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
    len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {df_cvd_DATE.LG[i].join(
    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(add_date_and_accuracy, axis=1)

1
Klart
2
Klart
3
Klart
4
Klart
5
Klart
6
Klart
1
Klart
2
Klart


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

# Adderar fält
Adderar fält till feature classes och importerar information från lednignar.txt.<br>
Lägger även till fält som kommer att styra de olika fältens egenskaper.

In [10]:
# Add data from LEDNIGNAR.txt
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

SKB_location = os.path.join(gdb, 'SKB_XYZ')

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    LG_date = cvd_DATE[LG.join("_00" + str(line))]
    fieldnames = ["_SKB_XYZ"]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")

    #if arcpy.Exists(SKB_path):
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True

        # Creating actual fields in the geodatabase and calculating values
        arcpy.management.AddField(fc_SKB, "LITTERA", "TEXT")
        arcpy.management.CalculateField(fc_SKB, "LITTERA", "".join(("'", littera, "'")))
        arcpy.management.AddField(fc_SKB, "LEDNINGSGATA", "LONG")
        arcpy.management.CalculateField(fc_SKB, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(fc_SKB, "RGDTM", "DATE")
        arcpy.management.CalculateField(fc_SKB, "RGDTM", "".join(("'", LG_date, "'")))
        arcpy.management.AddField(fc_SKB, "Ursprung", "TEXT")
        arcpy.management.CalculateField(fc_SKB, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(fc_SKB, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(fc_SKB, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(fc_SKB, "Insamlingsmetod", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(fc_SKB, "Matosakerhet_Plan", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(fc_SKB, "Matosakerhet_Hojd", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Matosakerhet_Hojd", int(matosak_hojd))

        dataset = SKB_location
        sr = arcpy.SpatialReference("SWEREF99_TM")
        arcpy.DefineProjection_management(dataset, sr)
        # Assigning domains to fields
        arcpy.management.AssignDomainToField(
            fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(
            fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")

    print("Klart")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
    len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {df_cvd_DATE.LG[i].join(
    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(add_date_and_accuracy, axis=1)


Doing LG040_1
Klart
Doing LG040_2
Klart
Doing LG040_3
Klart
Doing LG040_4
Klart
Doing LG040_5
Klart
Doing LG040_6
Klart
Doing LG661_1
Klart
Doing LG661_2
Klart


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

## Ändra data i fält:
Avrunda till en decimal, fixa rätt fältnamn och alias

In [11]:
import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
 
    fieldnames = ["_SKB_XYZ"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]
            if "AVSTAND_HORISONTELLT" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['AVSTAND_HORISONTELLT']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "AVSTAND_FAS" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_FAS",
                    "AVSTAND_FAS",
                    "AVSTAND_FAS")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['AVSTAND_FAS']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "DELTA_HOJD" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "DELTA_HOJD",
                    "DELTA_HOJD",
                    "DELTA_HOJD")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['DELTA_HOJD']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "RGDTM" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "RGDTM",
                    "RGDTM",
                    "RGDTM")

            if "MATOSAKERHET_PLAN" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "MATOSAKERHET_PLAN",
                    "Matosakerhet_Plan",
                    "Matosakerhet_Plan")

            if "MATOSAKERHET_HOJD" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "MATOSAKERHET_HOJD",
                    "Matosakerhet_Hojd",
                    "Matosakerhet_Hojd")

            if "LEDNINGSGATA" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "LEDNINGSGATA",
                    "LEDNINGSGATA",
                    "LEDNINGSGATA")


    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG040_1_SKB_XYZ
No RBX for LG040_1
Altering fields for LG040_2_SKB_XYZ
No RBX for LG040_2
Altering fields for LG040_3_SKB_XYZ
No RBX for LG040_3
Altering fields for LG040_4_SKB_XYZ
No RBX for LG040_4
Altering fields for LG040_5_SKB_XYZ
No RBX for LG040_5
Altering fields for LG040_6_SKB_XYZ
No RBX for LG040_6
Altering fields for LG661_1_SKB_XYZ
No RBX for LG661_1
Altering fields for LG661_2_SKB_XYZ
No RBX for LG661_2


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

# Slå ihop kantträd för alla ledningar till en featureclass

In [12]:
# Merge all to one
import datetime, arcpy, os
from pathlib import Path
import pandas as pd

all_location = os.path.join(gdb, 'SKB_XYZ')
all_merged_location = os.path.join(gdb, 'all')
all_merged_path = os.path.join(all_merged_location, all_merged_name)

def to_all_list(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    print(f"Adding {LG}_{line} to list")
    
    all_path = os.path.join(all_location, f"{LG}_{line}_SKB_XYZ")

    all_list.append(all_path)
    
    print(f"Done adding {LG}_{line} to list")

    
all_list = []

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(to_all_list, axis=1)

print("Doing merge")
arcpy.Merge_management(all_list, all_merged_path)


Adding LG040_1 to list
Done adding LG040_1 to list
Adding LG040_2 to list
Done adding LG040_2 to list
Adding LG040_3 to list
Done adding LG040_3 to list
Adding LG040_4 to list
Done adding LG040_4 to list
Adding LG040_5 to list
Done adding LG040_5 to list
Adding LG040_6 to list
Done adding LG040_6 to list
Adding LG661_1 to list
Done adding LG661_1 to list
Adding LG661_2 to list
Done adding LG661_2 to list
Doing merge


<Result 'C:\\Users\\semnve\\Desktop\\RBX_och_SKB\\SKB_2023_11_01\\SKB_2023_11_01.gdb\\all\\all_2023_11_01'>

# Klipp träffpunkter
Se till att använda aktuella data i fc_ledningsgator och fc_stationsområden

In [13]:
import arcpy
import os
import pandas as pd

fc_ledningsgator = os.path.join(gdb, "Gator2021")
fc_stationsomraden = os.path.join(gdb, "STATIONSOMRADE")

arcpy.env.overwriteOutput = True

##fd_ej_akuta = os.path.join(gdb, "ej_akuta")
fd_all = os.path.join(gdb, "all")
fd_traffpunkter = os.path.join(gdb, "traffpunkter")

fc_all = os.path.join(fd_all, f"all_{version}")
fc_traffpunkter = os.path.join(fd_traffpunkter, f"traffpunkter_{version}")
tmp_layer_traffpunkter = os.path.join(gdb, "tmp_layer_traffpunkter")

arcpy.management.Delete(tmp_layer_traffpunkter)
arcpy.management.MakeFeatureLayer(fc_all, tmp_layer_traffpunkter)


# Steg 1
# Inom fastbredd och stationsområden: ta bort alla inom 7 m horisontellt avstånd från fas
#
# 1.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 1.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 1.3 Av de valda, behåll punkter med avst_hori < 7 m
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                        selection_type = "SUBSET_SELECTION", 
                                        where_clause = '"AVSTAND_HORISONTELLT" < 7')
# 1.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Steg 2
# Inom fastbredd och stationsområden: ta bort alla med trädhöjd <= 4.5 m och avst_fas >= 5 m
#
# 2.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 2.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 2.3 Av de valda, behåll punkter med trädhöjd <= 4.5 OCH avst_fas >= 5
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                       selection_type = "SUBSET_SELECTION",
                                       where_clause = '"DELTA_HOJD" <= 4.5 AND "AVSTAND_FAS" >= 5')
# 2.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Kopiera ej_akuta till traffpunkter
arcpy.management.CopyFeatures(tmp_layer_traffpunkter, fc_traffpunkter)

# Ta bort det temporära lagret
arcpy.management.Delete(tmp_layer_traffpunkter)


<Result 'true'>

# Rensning onödig information
Rensar rådata från onödig information

Första cellen gör rensningen på de ej hopslagna featureklasserna dvs varje ledninge för sig

Andra cellen gör rensningen på featureklassen med alla ledningar hopslagna (hopslagningen görs under rubrik "Slå ihop kantträd för alla ledningar till en featureklass" ovan)

TODO: gör rensningen på enskilda ledningar innan hopslagningen görs. Kontrollera om rensningen även kan göras innan klippningen, eller om klippningen är beroende av några av de fält som här rensas bort

In [14]:
import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd


SKB_XYZ_fd = os.path.join(prj_dir, gdb, "SKB_XYZ")                                                                                            

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    
    fieldnames = ["_SKB_XYZ"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, fc_SKB)
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]

            if "GIS_ID" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "GIS_ID")
            if "MAX_TILLVAXT" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "MAX_TILLVAXT")
            if "created_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_user")
            if "created_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_date")
            if "last_edited_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_user")
            if "last_edited_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_date")
            if "ATGARDSAR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "ATGARDSAR")
            if "LEVERANTOR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTOR")
            if "LEVERANTORS_DATUM" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTORS_DATUM")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG040_1_SKB_XYZ
No RBX for LG040_1
Altering fields for LG040_2_SKB_XYZ
No RBX for LG040_2
Altering fields for LG040_3_SKB_XYZ
No RBX for LG040_3
Altering fields for LG040_4_SKB_XYZ
No RBX for LG040_4
Altering fields for LG040_5_SKB_XYZ
No RBX for LG040_5
Altering fields for LG040_6_SKB_XYZ
No RBX for LG040_6
Altering fields for LG661_1_SKB_XYZ
No RBX for LG661_1
Altering fields for LG661_2_SKB_XYZ
No RBX for LG661_2


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

In [15]:
# Rensar rådata från onödig information

import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    
    datasetnames = ["all"]
    fieldnames = [version]
    
    for dataset in datasetnames:
        print(dataset)
        for fieldname in fieldnames:
            fc_SKB = f"{dataset}_{fieldname}"
            if "all" in dataset:
                SKB_location = os.path.join(prj_dir,gdb,f"{dataset}")
            else:
                SKB_location = os.path.join(prj_dir,gdb,f"{dataset}")
            print(SKB_location)
            SKB_path = os.path.join(SKB_location, fc_SKB)
 
            # Activating workspace
            arcpy.env.workspace = SKB_path
            arcpy.env.overwriteOutput = True

            if arcpy.Exists(fc_SKB):
                SKB_XYZ_fc = fc_SKB
                print(f"Altering fields for {dataset}_{fieldname}")
                path_joined = os.path.join(SKB_location, f'{dataset}_{fieldname}')
                field_names = [f.name for f in arcpy.ListFields(SKB_path)]

                if "GIS_ID" in field_names:    
                        arcpy.management.DeleteField(
                            path_joined,
                            "GIS_ID")
                if "MAX_TILLVAXT" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "MAX_TILLVAXT")
                if "created_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_user")
                if "created_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_date")
                if "last_edited_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_user")
                if "last_edited_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_date")
                if "ATGARDSAR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "ATGARDSAR")
                if "LEVERANTOR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTOR")
                if "LEVERANTORS_DATUM" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTORS_DATUM")
                
                dataset = SKB_location
                sr = arcpy.SpatialReference("SWEREF99_TM")
                arcpy.DefineProjection_management(dataset, sr)
                # Assigning domains to fields
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01
all
C:\Users\semnve\Desktop\RBX_och_SKB\SKB_2023_11_01\SKB_2023_11_01.gdb\all
Altering fields for all_2023_11_01


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

## Ta bort feature datasets
Rensar bort onödiga featureklasser från leveransdatabasen.
TODO: går det att göra en bättre "mall-databas" om sådan används, eller låta bli att alls skapa de onödiga featureklasserna?

In [16]:
import arcpy
 
# Set environment settings
arcpy.env.workspace = gdb

features = [
    "ej_akuta",
    "akuta",
    "clean_RBX_points",
    "mittfas",
    "SKB_XYmZ",
    "SKB_XYZ",
    "traffpunkter",
    "Wires",
    "Gator2021",
    "SKB_template",
    "STATIONSOMRADE"]

for feature in features:
    arcpy.management.Delete(os.path.join(gdb, feature))